In [1]:
import os
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from keras.utils import to_categorical

2024-07-13 22:48:50.637940: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


---

In [2]:
DATASET_PATH = os.path.join("..", "..", "data", "star_types.csv")

In [3]:
df = pd.read_csv(DATASET_PATH)
df.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [4]:
# --------------------------------------------------------------------------------------
label_encoder_color = LabelEncoder()
df['Star color'] = label_encoder_color.fit_transform(df['Star color'])
label_encoder_class = LabelEncoder()
df['Spectral Class'] = label_encoder_class.fit_transform(df['Spectral Class'])

df["Temperature (K)"] = df["Temperature (K)"].astype(float)
df["Luminosity(L/Lo)"] = (df["Luminosity(L/Lo)"] * 1e5).astype(float)
df["Radius(R/Ro)"] = (df["Radius(R/Ro)"] * 1e3).astype(float)
df["Absolute magnitude(Mv)"] = (df["Absolute magnitude(Mv)"] * 1e2).astype(float)
# --------------------------------------------------------------------------------------
df["Star type"] = df["Star type"].astype(int)
# --------------------------------------------------------------------------------------
scaler = StandardScaler()
df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)"]] = scaler.fit_transform(
    df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)"]]
)
# --------------------------------------------------------------------------------------
X = df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)", "Star color", "Spectral Class"]].values
y = to_categorical(df["Star type"].values)

In [5]:
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.2, random_state=42)

train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

---

In [6]:
from basic_conversions import matrix_to_wfloat

In [7]:
print(matrix_to_wfloat(train_X[:5]))
print("--------------------")
print(matrix_to_wfloat(train_Y[:5]))

[[3618502788666131213697322783095070105623107215331596699973092056135871290719
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871561438
  946440 10000000 5000000]
 [3618502788666131213697322783095070105623107215331596699973092056135871269738
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871561053
  1127211 10000000 5000000]
 [822715
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871560966
  650546 0 1000000]
 [3618502788666131213697322783095070105623107215331596699973092056135871230609
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871561173
  1243284 10000000 5000000]
 [1321220
  3618502

---

In [8]:
import basic_model_sample
from basic_model_sample import DenseLayer, Sequential, SGD

In [22]:
layers = [
    DenseLayer(input_shape=train_X.shape[1], output_shape=7, activation="ReLU"), 
    DenseLayer(output_shape=4, activation="ReLU"), 
    DenseLayer(output_shape=8, activation="ReLU"), 
    DenseLayer(output_shape=train_Y.shape[1], activation="ReLU"), 
]

network = Sequential(layers, SGD(learning_rate=0.1))
network.build()
print("num params: ", network.num_params())

# print(f"Num params: {network.num_params()}")

network.train(train_X, train_Y, epochs=100, batch_size=1, verbose=True)

predictions = network.forward(train_X)
print(f"prediction \n{predictions}")
loss = network.mse_loss(predictions, train_Y)
print(f"Final loss on test set: {loss}")

num params:  175
Epoch 1, Loss: 0.14890554761457692
Epoch 11, Loss: 0.15136967779975471
Epoch 21, Loss: 0.15265155158983662
Epoch 31, Loss: 0.15782227938997767
Epoch 41, Loss: 0.15118181566362957
Epoch 51, Loss: 0.15721655555842987
Epoch 61, Loss: 0.15104416284704586
Epoch 71, Loss: 0.1490751248726
Epoch 81, Loss: 0.15109730793266632
Epoch 91, Loss: 0.15560623477140656
prediction 
[[0.24002216 0.13684908 0.         0.09177647 0.         0.04381452]
 [0.24002216 0.13684908 0.         0.09177647 0.         0.04381452]
 [0.24002216 0.13684908 0.         0.09177647 0.         0.04381452]
 ...
 [0.24002216 0.13684908 0.         0.09177647 0.         0.04381452]
 [0.24002216 0.13684908 0.         0.09177647 0.         0.04381452]
 [0.24002216 0.13684908 0.         0.09177647 0.         0.04381452]]
Final loss on test set: 0.15263398168513473


In [23]:
network.serialize()

'4, 3, 6, 7, 17140, 3618502788666131213697322783095070105623107215331596699973092056135872011775, 2776, 4143, 3618502788666131213697322783095070105623107215331596699973092056135872014391, 8251, 3618502788666131213697322783095070105623107215331596699973092056135872010520, 7, 7983, 3618502788666131213697322783095070105623107215331596699973092056135872012879, 1664, 3618502788666131213697322783095070105623107215331596699973092056135872018333, 1893, 3618502788666131213697322783095070105623107215331596699973092056135872019712, 3618502788666131213697322783095070105623107215331596699973092056135872014204, 7, 3618502788666131213697322783095070105623107215331596699973092056135872018764, 3618502788666131213697322783095070105623107215331596699973092056135872014830, 4457, 5017, 10799, 8692, 3618502788666131213697322783095070105623107215331596699973092056135872019837, 7, 3618502788666131213697322783095070105623107215331596699973092056135872015049, 3618502788666131213697322783095070105623107215331596

In [24]:
with open('model_weights.save', '+w') as f: 
    f.write(network.serialize())